In [19]:
import numpy as np
import pandas as pd

In [20]:
pop = {
    'NonTB': np.array([0.58453037, 0.10199624]),
    'TBLike': np.array([0.01807826, 0.00315452]),
    'LTBI': np.array([0.2264542, 0.0738548]),
    'Asym': np.array([0.00135638, 0.00108361]),
    'Sym': np.array([0.00065895, 0.00053224]),
}

n = sum([v.sum() for v in pop.values()])

coverage = 0.1
pop = {k: v / n * coverage  for k, v in pop.items()}
pop

{'NonTB': array([0.05777707, 0.01008167]),
 'TBLike': array([0.00178692, 0.0003118 ]),
 'LTBI': array([0.02238354, 0.00730007]),
 'Asym': array([0.00013407, 0.00010711]),
 'Sym': array([6.51329722e-05, 5.26085031e-05])}

In [21]:
def sym():
    return {
        'NonTB': 0, 'TBLike': 1, 'LTBI': 0, 
        'Asym': 0, 'Sym': 1
    }


def xpert(sens=0.92, spec=0.99):
    return {
        'NonTB': (1 - spec), 'TBLike': (1 - spec),
        'LTBI': (1 - spec), 
        'Asym': sens, 'Sym': sens
    }


def cxr(sens=0.98, spec=0.75):
    return {
        'NonTB': (1 - spec), 'TBLike': (1 - spec), 'LTBI': (1 - spec),
        'Asym': sens, 'Sym': sens
    }


def vul():
    return {
        'NonTB': np.array([0, 1]), 'TBLike': np.array([0, 1]), 'LTBI': np.array([0, 1]),
        'Asym': np.array([0, 1]), 'Sym': np.array([0, 1])
    }



In [31]:
def And(tool1, tool2):
    return {k: v1 * tool2[k] for k, v1 in tool1.items()}

def Not(tool):
    return {k: 1 - v for k, v in tool.items()}

def Or(tool1, tool2):
    return {k: 1 - (1 - v1) * (1 - tool2[k]) for k, v1 in tool1.items()}

def Plus(tool1, tool2):
    return {k: v1 + tool2[k] for k, v1 in tool1.items()}

def run(pop, alg):
    yld = {k: n * alg[k] for k, n in pop.items()}
    
    n_yld = sum([v.sum() for v in yld.values()])
    n_tp = yld['Asym'].sum() + yld['Sym'].sum()
    
    n_tpt = yld['LTBI'].sum()
    return {
        'Yields': n_yld * 1e5,
        'TP': n_tp * 1e5,
        'PPV': n_tp / n_yld,
        'OnTPT': n_tpt * 1e5,
        #'Pos': yld
    }

In [32]:
Sym, CXR, Xpert, Vul = sym(), cxr(), xpert(), vul()

In [33]:
run(pop, And(Or(Sym, CXR), Xpert))

{'Yields': 59.061076886688845,
 'TP': 32.57676371257132,
 'PPV': 0.5515775436176217,
 'OnTPT': 7.420903618650355}

In [34]:
run(pop, And(Sym, Xpert))

{'Yields': 12.93093956736584,
 'TP': 10.832215733767688,
 'PPV': 0.8376974988813065,
 'OnTPT': 0.0}

In [35]:
run(pop,  
    And(
        Plus(Sym, And(And(Vul, Not(Sym)), CXR)), 
        Xpert
    ))

{'Yields': 26.933222636439407,
 'TP': 20.489062538595334,
 'PPV': 0.7607356466461083,
 'OnTPT': 1.825018073300162}

In [36]:
run(pop,  
    And(
        Or(Sym, Vul), 
        Xpert
    ))

{'Yields': 40.16660993539815,
 'TP': 20.686141044816306,
 'PPV': 0.5150083882629577,
 'OnTPT': 7.300072293200648}

In [39]:
run(pop, Xpert)

{'Yields': 132.66161119352867,
 'TP': 33.020529997853025,
 'PPV': 0.2489079523516581,
 'OnTPT': 29.68361447460142}